In [78]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.pylab import subplots

## Broken Road


Given a rod of unit-length, broken independently and randomly at two places, what is the probability that you can assemble the three remaining pieces into a triangle?

In [10]:
x,y = np.random.rand(2,1000)
a,b,c = x,(y-x),1-y
s = (a+b+c)/2
np.mean((s>a) & (s>b) & (s>c) & (y>x))

0.13400000000000001

## Conditional Probabilities

Three coins, 10p, 20p and 50p are tossed. The values of the coins that land heads up are totaled. What is the expected total given that two coins have landed heads-up.

In [16]:
d = pd.DataFrame(columns=['X10','X20','X50'])
d.X10 = np.random.randint(0,2,1000)
d.X10 = np.random.randint(0,2,1000)
d.X20 = np.random.randint(0,2,1000)
d.X50 = np.random.randint(0,2,1000)

In [18]:
grp=d.groupby(d.eval('X10+X20+X50'))
grp.get_group(2).eval('10*X10+20*X20+50*X50').mean()

52.777777777777779

## Simulating a fair dice

In [21]:
u= np.random.rand(100)
df = pd.DataFrame(data=u,columns=['u'])
labels = [1,2,3,4,5,6]
df['v']=pd.cut(df.u,np.linspace(0,1,7),include_lowest=True,labels=labels)

In [24]:
df.groupby('v').count()

,u
v,
1,21
2,18
3,14
4,19
5,16
6,12


## Generating an exponential distribution

In [48]:
import scipy.stats
alpha = 1
nsamp = 1000 # num of samples

u=scipy.stats.uniform(0,1)
Finv=lambda u: 1/alpha*np.log(1/(1-u))
v = list(map(Finv,u.rvs(nsamp)))

In [63]:
plt.hist(v,bins=30)
plt.show()

## Rejection method for densities without continuous inverse

In [65]:
 x = np.linspace(0.001,15,100)
f= lambda x: np.exp(-(x-1)**2/2./x)*(x+1)/12.
fx = f(x)
M=0.3 # scale factor
u1 = np.random.rand(10000)*15 
u2 = np.random.rand(10000)  # uniform random samples
idx,= np.where(u2<=f(u1)/M) # rejection criterion
v = u1[idx]                 # keep these only

## Coin tossing

In [70]:
from scipy.stats import bernoulli
p_true=1/2.0
fp=bernoulli(p_true) 
xs = fp.rvs(100) # generate some samples
xs[:30] # see first 30 samples

array([0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 0])

In [71]:
import sympy
x,p,z=sympy.symbols('x p z', positive=True)
phi=p**x*(1-p)**(1-x) # distribution function
L=np.prod([phi.subs(x,i) for i in xs]) # likelihood function
print(L)

p**52.0*(-p + 1)**48


In [72]:
logL=sympy.expand_log(sympy.log(L))
sol,=sympy.solve(sympy.diff(logL,p),p)
print(sol)

0.520000000000000


## Linear Regression

In [79]:
from sklearn.linear_model import LinearRegression
X = np.arange(10) # create some data
Y = X+np.random.randn(10) # linear with noise

In [85]:
plt.plot(X,Y,"x")
plt.show()

In [86]:
lr=LinearRegression() # create model

In [91]:
X,Y = X.reshape((-1,1)), Y.reshape((-1,1)) # reshape as columns
lr.fit(X,Y)
lr.coef_

array([[ 0.94082165]])

In [92]:
xi = np.linspace(0,10,15)
xi = xi.reshape((-1,1)) 
yp = lr.predict(xi)

Multilinear Regression

In [94]:
X=np.random.randint(20,size=(10,2))
Y=X.dot([1, 3])+1 + np.random.randn(X.shape[0])*20
lr=LinearRegression()
lr.fit(X,Y)
lr.coef_

array([ 1.6151495 ,  2.98940165])

Polynomial Regression

In [96]:
from sklearn.preprocessing import PolynomialFeatures
X = np.arange(10).reshape(-1,1) # create some data
Y = X+X**2+X**3+ np.random.randn(*X.shape)*80

In [97]:
qfit = PolynomialFeatures(degree=2) # quadratic
Xq = qfit.fit_transform(X)

In [98]:
lr=LinearRegression() # create linear model
qr=LinearRegression() # create quadratic model
lr.fit(X,Y) # fit linear model
qr.fit(Xq,Y) # fit quadratic model
lp = lr.predict(xi)
qp = qr.predict(qfit.fit_transform(xi))